In [2]:
import re
from collections import Counter
from itertools import permutations
from typing import List

import black
import jupyter_black
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> List[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))

In [6]:
# Day 1: Report Repair
numbers = [int(x) for x in open("2020/1.txt").read().splitlines()]
for a, b in permutations(numbers, 2):
    if a + b == 2020:
        break
print(f"Part 1: {a*b}")  # 471019

for a, b, c in permutations(numbers, 3):
    if a + b + c == 2020:
        break
print(f"Part 2: {a*b*c}")  # 103927824

Part 1: 471019
Part 2: 103927824


In [24]:
# Day 2: Password Philosophy
def valid(line):
    lower, upper, char, password = parse("{:d}-{:d} {}: {}", line)
    return lower <= password.count(char) <= upper


def valid2(line):
    first, second, char, password = parse("{:d}-{:d} {}: {}", line)
    return ((password[first - 1] == char) + (password[second - 1] == char)) == 1


lines = open("2020/2.txt").read().splitlines()
print(f"Part 1: {sum(valid(line) for line in lines)}") # 398
print(f"Part 2: {sum(valid2(line) for line in lines)}") # 562

Part 1: 398
Part 2: 562


In [50]:
# Day 3: Toboggan Trajectory
def downhill(lines, right=3, skip_every_other_line=False):
    pos, trees = right, 0
    skip = True
    for line in lines[1:]:
        if skip_every_other_line:
            if skip:
                skip = False
                continue
            else:
                skip = True
        if line[pos] == "#":
            trees += 1
        pos = (pos + right) % len(line)
    return trees


def run(lines, slopes):
    result = 1
    for slope in slopes:
        result *= downhill(lines, slope[0], slope[1])
    return result


lines = open("2020/3.txt").read().splitlines()
slopes = ((1, False), (3, False), (5, False), (7, False), (1, True))
print(f"Part 1: {downhill(lines)}")  # 189
print(f"Part 2: {run(lines, slopes)}")  # 1718180100

Part 1: 189
Part 2: 1718180100


In [172]:
# Day 4: Passport Processing
def all_fields_present(passport):
    FIELDS_NEEDED = {"byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"}
    fields = passport.split(" ")
    fields = {field.split(":")[0]: field.split(":")[1] for field in fields}
    return all(field in fields.keys() for field in FIELDS_NEEDED)


def valid(passport):
    def check_height(height):
        unit = height[-2:]
        value = first(ints(height))
        if unit == "cm":
            return 150 <= value <= 193
        elif unit == "in":
            return 59 <= value <= 76
        else:
            return False

    EYE_COLORS = {"amb", "blu", "brn", "grn", "gry", "hzl", "oth"}
    rules = {
        "byr": lambda year: 1920 <= int(year) <= 2002,
        "iyr": lambda year: 2010 <= int(year) <= 2020,
        "eyr": lambda year: 2020 <= int(year) <= 2030,
        "hgt": check_height,
        "hcl": lambda color: re.search("#[0-9a-f]{6}", color) is not None,
        "ecl": lambda color: color in EYE_COLORS,
        "pid": lambda number: re.search("^[0-9]{9}$", number) is not None,
        "cid": lambda x: True,
    }
    fields = passport.split(" ")
    fields = {field.split(":")[0]: field.split(":")[1] for field in fields}
    if all(rules[key](value) for key, value in fields.items()):
        return True


passports = open("2020/4.txt").read().strip().split("\n\n")
passports = [p.replace("\n", " ") for p in passports]
passports = [p for p in passports if all_fields_present(p)]
print(f"Part 1: {len(passports)}")
passports = [p for p in passports if valid(p)]
print(f"Part 2: {len(passports)}")

Part 1: 170
Part 2: 103


In [199]:
# Day 5: Binary Boarding
def seat_id(boarding_pass):
    row = boarding_pass[:7].replace("F", "0").replace("B", "1")
    column = boarding_pass[7:].replace("L", "0").replace("R", "1")
    return int(row, 2) * 8 + int(column, 2)


boarding_passes = open("2020/5.txt").read().splitlines()
seat_ids = sorted(seat_id(boarding_pass) for boarding_pass in boarding_passes)
print(f"Part 1: {max(seat_ids)}")
my_seat_id = first(
    (higher - 1) for lower, higher in zip(seat_ids, seat_ids[1:]) if higher - lower == 2
)
print(f"Part 2: {my_seat_id}")

Part 1: 991
Part 2: 534


In [41]:
# Day 6: Custom Customs
def unique_answers(groups):
    total = 0
    for group in groups:
        answers = set()
        for person in group:
            for question in person:
                answers.add(question)
        total += len(answers)
    return total


def all_yes_answers(groups):
    total = 0
    for group in groups:
        group_answers = Counter()
        for person in group:
            for question in person:
                group_answers += Counter(question)
        for answer, count in group_answers.items():
            if count == len(group):
                total += 1
    return total


groups = open("2020/6.txt").read().strip().split("\n\n")
groups = [group.splitlines() for group in groups]

print(f"Part 1: {unique_answers(groups)}")
print(f"Part 2: {all_yes_answers(groups)}")

Part 1: 6683
Part 2: 3122
